In [ ]:
import torch
from transformers import AutoTokenizer,OlmoeForCausalLM, OlmoeConfig
torch.set_printoptions(profile='simple')
device = torch.device("cuda:0")
model_path = "shuyuej/OLMoE-1B-7B-0924-GPTQ" #"allenai/OLMoE-1B-7B-0924" 
config = OlmoeConfig.from_pretrained(model_path,num_experts_per_tok=4)
model = OlmoeForCausalLM.from_pretrained(model_path,config=config,
                                         device_map={"": device},
                                         torch_dtype=torch.float16,
                                         )
tokenizer = AutoTokenizer.from_pretrained(model_path)
model.eval()

🚨 `use_probabilistic_routing` is part of OlmoeModel.forward's signature, but not documented. Make sure to add it to the docstring of the function in /home/tkode/Desktop/transformers/src/transformers/models/olmoe/modeling_olmoe.py.
🚨 `prob_routing_temp` is part of OlmoeModel.forward's signature, but not documented. Make sure to add it to the docstring of the function in /home/tkode/Desktop/transformers/src/transformers/models/olmoe/modeling_olmoe.py.
🚨 `use_probabilistic_routing` is part of OlmoeForCausalLM.forward's signature, but not documented. Make sure to add it to the docstring of the function in /home/tkode/Desktop/transformers/src/transformers/models/olmoe/modeling_olmoe.py.
🚨 `prob_routing_temp` is part of OlmoeForCausalLM.forward's signature, but not documented. Make sure to add it to the docstring of the function in /home/tkode/Desktop/transformers/src/transformers/models/olmoe/modeling_olmoe.py.


/home/tkode/miniconda3/envs/llm/lib/python3.9/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:411: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, qweight, scales, qzeros, g_idx, bits, maxq):
/home/tkode/miniconda3/envs/llm/lib/python3.9/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:419: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/tkode/miniconda3/envs/llm/lib/python3.9/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.
CUDA extension not installed.


model.safetensors:   0%|          | 0.00/3.92G [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

OlmoeForCausalLM(
  (model): OlmoeModel(
    (embed_tokens): Embedding(50304, 2048, padding_idx=1)
    (layers): ModuleList(
      (0-15): 16 x OlmoeDecoderLayer(
        (self_attn): OlmoeSdpaAttention(
          (q_norm): OlmoeRMSNorm((2048,), eps=1e-05)
          (k_norm): OlmoeRMSNorm((2048,), eps=1e-05)
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): OlmoeSparseMoeBlock(
          (experts): ModuleList(
            (0-63): 64 x OlmoeMLP(
              (act_fn): SiLU()
              (down_proj): QuantLinear()
              (gate_proj): QuantLinear()
              (up_proj): QuantLinear()
            )
          )
          (gate): QuantLinear()
        )
        (input_layernorm): OlmoeRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): OlmoeRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): OlmoeRMSNorm((2048,), eps=1e-05)
    (rotary_emb): OlmoeRotaryEmb

In [2]:
if "inputs" in globals():
    del inputs

prompt = 'Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, there is much inventive camera work. There is an innocent American who gets emotionally involved with a woman he doesn\'t really understand, and whose naivety is all the more striking in contrast with the natives.<br /><br />But I\'d have to say that The Third Man has a more well-crafted storyline. Zentropa is a bit disjointed in this respect. Perhaps this is intentional: it is presented as a dream/nightmare, and making it too coherent would spoil the effect. <br /><br />This movie is unrelentingly grim--"noir" in more than one sense; one never sees the sun shine. Grim, but intriguing, and frightening.' +' Is the prior text positive or negative? Give us a rating out of 10'
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# prob_routing_temp cannot be lower than about 0.0001. 
# 0.00001 causes cuda assert


with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        use_probabilistic_routing=True,
        prob_routing_temp=0.1,
        #output_router_logits=True,
        #return_dict=True
    )

print(tokenizer.decode(outputs[0]))

Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, there is much inventive camera work. There is an innocent American who gets emotionally involved with a woman he doesn't really understand, and whose naivety is all the more striking in contrast with the natives.<br /><br />But I'd have to say that The Third Man has a more well-crafted storyline. Zentropa is a bit disjointed in this respect. Perhaps this is intentional: it is presented as a dream/nightmare, and making it too coherent would spoil the effect. <br /><br />This movie is unrelentingly grim--"noir" in more than one sense; one never sees the sun shine. Grim, but intriguing, and frightening. Is the prior text positive or negative? Give us a rating out of 10.http://www.thetwilightproject.com/2011/01/third-man-and-zentropa-review.htmlnoreply@blogger.com (David Fiore)0tag:blogger.com,1999:blog
